<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/How_to_Get_Puerto_Rico_Real_Estate_Data_%7C_Python_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Get Puerto Rico Real Estate Data | Python Tutorial
<i>Python tutorial to get Puerto Rico for-sale and for-rent (short-term and long-term) listings</i>
<br>
<br>
![link text](https://drive.google.com/uc?id=10oWFppsYIkOynHBEy0nXuxS5NuVDKO2u)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | Rapid API |
| Input Datasets        | [Puerto Rico Real Estate API](https://bit.ly/3uLpbvt) |
| Output Datasets       | CSV file, Map HTML |
| Input Data Source     | JSON file |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 12th July 2022 | Ariel Herrera | Create notebook. |

## Background 🤓
- **Overview**: This notebook retrieves real estate property data using the [Puerto Rico Real Estate API](https://bit.ly/3uLpbvt). This includes properties for-sale and properties for rent (short-term and long-term).
- **Purpose**: This notebook is a Python tutorial to query the [Puerto Rico Real Estate API](https://bit.ly/3uLpbvt). Below are examples on how to extract listings data for towns in Puerto Rico.
- **Background**: Property data for Puerto Rico is not listed on the MLS (Multiple Listing Services). Properties for-sale and for-rent are listed by agents or by property owners on websites like [Clasificados Online](https://www.clasificadosonline.com/). The data is not easily consumable. This makes it difficult to *analyze the performance of real estate markets* across the country of Puerto Rico. Therefore, the API was created to web scrape the data and make it programmatically available to analyze.
- **Example Use Case(s)**:
  - **Investors** - Identify median household prices and median rent prices within a town. Determine cash flow.
  - **Agents** - Provide trend analysis statistics such as days on market (DOM), inventory supply, and median home price by property type across towns.
  - **Owner-occupied buyers** - View all properties for-sale in a single spreadsheet.
  - **Expats** - View all properites for-rent in a single spreadsheet. Perform clustering to group properties by similar features.
- **Dataset**: Listings from [Clasificados Online](https://www.clasificadosonline.com/)

## Getting Started ✅
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions
  - Create an account with [RapidAPI.com](rapidapi.com)
  - Copy your [RapidAPI key](https://rapidapi.com/blog/api-glossary/api-key/)
  - Store your RapidAPI key into an `api_key.csv` file OR replace the `rapid_api_key` variable <string> with your own RapidAPI key
  - Run notebook -> Runtime -> Run all

## Useful Resources 📑
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Clasificados Online](https://www.clasificadosonline.com/)
- [Puerto Rico Real Estate API](https://bit.ly/3uLpbvt)
- [Puerto Rico Towns](https://bit.ly/3RweCGr)

## Questions / Feedback❓
- [Contact AnalyticsAriel](https://www.analyticsariel.com/contact-me)

## <font color="blue">Install Packages</font>

In [1]:
!pip install keplergl -q # visualization

## <font color="blue">Imports</font>

In [2]:
from google.colab import drive, files, output # specific to Google Colab
import requests
import time

# data wrangling
import pandas as pd
import geopandas as gpd

# visualization
import plotly.express as px
from keplergl import KeplerGl

# settings
output.enable_custom_widget_manager() # enables to view charts in Google Colab

## <font color="blue">Functions</font>

In [3]:
def get_for_sale_listings(rapid_api_key, 
                          town_id,
                          page=1, # default first page
                          property_type=None,
                          low_price_range=None,
                          high_price_range=None,
                          bedrooms=None,
                          repo=0, # default set flag off
                          opt=0, # default set flag off
                          all_pages=False): # get data for all pages
  """
  Search for property for-sale listings in a geographical area. 
  Returns the property features, image url, and coordinates.

  Parameters
  ----------
  @rapid_api_key [string]: Key to access data from Rapid API
  @town_id [string]: 
  @page [string]: Page number
  @property_type [string]: 
    apartamento|apartamento/walkup|casa|comercial|finca|multifamiliar|solar
  @low_price_range [string]: Minimum price
  @high_price_range [string]: Maximum price
  @bedrooms [string]: Exact number of bedrooms
  @repo [number]: Flag for reposessed properties
  @opt [number]: Flag for options filter

  Returns
  -------
  [json] API response

  """

  #####################
  #   REQUEST DATA    #
  #####################

  # parameters
  if bedrooms != None:
    bedrooms = str(int(bedrooms))

  # request
  url = "https://puerto-rico-real-estate.p.rapidapi.com/property_search"

  querystring = {
    "town_id":town_id, # required
    "page":str(page),
    "property_type": property_type,
    "low_price_range": str(int(low_price_range)),
    "high_price_range": str(int(high_price_range)),
    "bedrooms": bedrooms,
    "repo": int(repo),
    "opt": int(opt),
  }

  headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "puerto-rico-real-estate.p.rapidapi.com"
  }

  response = requests.request("GET", url, headers=headers, params=querystring)

  #########################
  #   RESPONSE CONTENT    #
  #########################
  
  # get contents of response
  if response.status_code == 200:
    print('API request successful!')
  else:
    print('WARNING: API request unsuccessful!')

  # get total number of pages
  max_page = response.json()['meta']['max_page']

  # get request for all pages
  if (all_pages == True) and (max_page > 1):
    # create list with first page results
    response_list = [response]

    start_time = time.time()
    # iterate through each page
    print('Retrieved data for page: 1')
    for i in range(2, max_page + 1):
      print('Requesting data for page:', str(i))

      # set new page
      querystring['page'] = i

      # get response
      response = requests.request("GET", url, headers=headers, params=querystring)

      # add response to list
      response_list.append(response)

    print("--- %s seconds ---" % (time.time() - start_time))

    return response_list

  return requests.request("GET", url, headers=headers, params=querystring)

In [4]:
def transform_listings_to_df(response_list):
  return pd.concat([pd.DataFrame(r.json()['properties']) for r in for_sale_response_list])

In [5]:
def get_kepler_map_config():
  return {'config': {'mapState': {'bearing': 0,
   'dragRotate': False,
   'isSplit': False,
   'latitude': 18.455909,
   'longitude': -66.06917085,
   'pitch': 0,
   'zoom': 13},
  'mapStyle': {'mapStyles': {},
   'styleType': 'dark',
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'topLayerGroups': {},
   'visibleLayerGroups': {'3d building': False,
    'border': False,
    'building': True,
    'label': True,
    'land': True,
    'road': True,
    'water': True}},
  'visState': {'animationConfig': {'currentTime': None, 'speed': 1},
   'filters': [],
   'interactionConfig': {'brush': {'enabled': False, 'size': 0.5},
    'coordinate': {'enabled': False},
    'geocoder': {'enabled': False},
    'tooltip': {'compareMode': False,
     'compareType': 'absolute',
     'enabled': True,
     'fieldsToShow': {'data_1': [{'format': None, 'name': 'street_address'},
       {'format': None, 'name': 'bedrooms'},
       {'format': None, 'name': 'bathrooms'},
       {'format': None, 'name': 'list_price'},
       {'format': None, 'name': 'property_type'}]}}},
   'layerBlending': 'normal',
   'layers': [{'config': {'color': [18, 147, 154],
      'columns': {'altitude': None, 'lat': 'lat', 'lng': 'lon'},
      'dataId': 'data_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'Point',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': '2nbr5t',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'num_bedrooms',
       'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}},
    {'config': {'color': [221, 178, 124],
      'columns': {'altitude': None, 'lat': 'latitude', 'lng': 'longitude'},
      'dataId': 'data_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': False,
      'label': 'Point',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': 'cxqq17x',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'num_bedrooms',
       'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}},
    {'config': {'color': [146, 38, 198],
      'columns': {'lat0': 'lat',
       'lat1': 'latitude',
       'lng0': 'lon',
       'lng1': 'longitude'},
      'dataId': 'data_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': False,
      'label': ' ->  arc',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'opacity': 0.8,
       'sizeRange': [0, 10],
       'targetColor': None,
       'thickness': 2}},
     'id': '2d7ikce',
     'type': 'arc',
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'config': {'color': [136, 87, 44],
      'columns': {'alt0': None,
       'alt1': None,
       'lat0': 'lat',
       'lat1': 'latitude',
       'lng0': 'lon',
       'lng1': 'longitude'},
      'dataId': 'data_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': False,
      'label': ' ->  line',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'elevationScale': 1,
       'opacity': 0.8,
       'sizeRange': [0, 10],
       'targetColor': None,
       'thickness': 2}},
     'id': '7hkvr37',
     'type': 'line',
     'visualChannels': {'colorField': None,
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear'}},
    {'config': {'color': [255, 153, 31],
      'columns': {'geojson': 'geometry'},
      'dataId': 'data_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'data_1',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'ColorBrewer',
        'colors': ['#ffffb2',
         '#fed976',
         '#feb24c',
         '#fd8d3c',
         '#f03b20',
         '#bd0026'],
        'name': 'ColorBrewer YlOrRd-6',
        'type': 'sequential'},
       'elevationScale': 5,
       'enable3d': False,
       'enableElevationZoomFactor': True,
       'filled': True,
       'heightRange': [0, 500],
       'opacity': 0.8,
       'radius': 20,
       'radiusRange': [0, 50],
       'sizeRange': [0, 10],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'strokeOpacity': 0.8,
       'stroked': False,
       'thickness': 0.5,
       'wireframe': False}},
     'id': 'xetjzy',
     'type': 'geojson',
     'visualChannels': {'colorField': {'name': 'list_price_int',
       'type': 'integer'},
      'colorScale': 'quantile',
      'heightField': None,
      'heightScale': 'linear',
      'radiusField': None,
      'radiusScale': 'linear',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}}],
   'splitMaps': []}},
 'version': 'v1'}

## <font color="blue">Locals & Constants</font>

In [6]:
########################################################################
# OPTIONAL                                                             #
# 1) Create a file called 'api_keys.csv' to store your Rapid API key   #
# 2) Read in the file                                                  #
# 3) Set the Rapid API key to the rapid_api_key variable               #
########################################################################

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/Colab Data/input/' # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

### <font color="green">Section #1 - API Requests</font> 🇵🇷
This section will cover how to make API requests to the [Puerto Rico Real Estate API](https://bit.ly/3cexfP2). It demonstrates how to modify your search based on different parameters.

#### <font color="purple">1. Properties For-Sale</font> 🏘

Step #1 - <b>Select a town</b> to retrieve for-sale listings. View list of [Puerto Rico Towns](https://bit.ly/3RweCGr) including size and population.

In [8]:
# select a town ID 
town_id = 'San Juan - Condado-Miramar'

# get data for town
for_sale_response = get_for_sale_listings(
    rapid_api_key=rapid_api_key, 
    town_id=town_id,
    low_price_range=5000, # minimum price of property
    high_price_range=500000 # maximum price of property
)

API request successful!


Step #2 - View response in text <string> format

In [9]:
# view raw text response before we transform into a table
for_sale_response.text

'{"properties":[{"street_address":"Cond. Puerta de la Bahía, CASH 3  hab...","bedrooms":"3 Cuartos","bathrooms":"1 Baños","list_price":"$110,000","property_type":"Apt/WalkUp","town":"Condominio-Puerta De La Bahia , San Juan - Santurce","image_url":"https://imgcache.clasificadosonline.com//PP/FS/2022/6/6/06062022071345gmki5fqp.jpeg","url":"https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4675687","lat":"18.4473458","lon":"-66.0791937"},{"street_address":"COND DIPLOMAT CONDADO","bedrooms":"1 Cuartos","bathrooms":"1 Baños","list_price":"$222,000","property_type":"Apartamento","town":"Condominio-Diplomat , San Juan - Condado-Miramar","image_url":"https://imgcache.clasificadosonline.com//PP/FS/2022/7/14/071420221901420teghxcj.jpg","url":"https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4684279","lat":"18.456037","lon":"-66.07258"},{"street_address":"Condado Lagoon Villas","bedrooms":"1 Cuartos","bathrooms":"1 Baños","list_price":"$400,000","property_type":"Apartament

Step #3 - View attributes of the response including the `listing_url` and number of pages `max_pages`

In [10]:
print('Response keys:', list(for_sale_response.json().keys()))

# view attributes in 'meta' field
for key, value in for_sale_response.json()['meta'].items():
  print(key, ':', value)

Response keys: ['properties', 'meta']
listing_url : https://www.clasificadosonline.com/UDREListing.asp?RESPueblos=San+Juan+-+Condado-Miramar&Category=%25&LowPrice=5000&HighPrice=500000&Bedrooms=%25&Area=&BtnSearchListing=See+Listing&redirecturl=%2Fudrelistingmap.asp&IncPrecio=1&offset=0
current_page : 1
max_page : 3


Step #4 - Get data for ALL pages with listings

In [11]:
# get data for town
for_sale_response_list = get_for_sale_listings(
    rapid_api_key=rapid_api_key, 
    town_id=town_id,
    low_price_range=5000, # minimum price of property
    high_price_range=500000, # maximum price of property
    all_pages=True # *get data for ALL pages*
)

API request successful!
Retrieved data for page: 1
Requesting data for page: 2
Requesting data for page: 3
--- 5.283110618591309 seconds ---


Step #5 - Transform text into a dataframe (table with rows and columns)

In [12]:
# transform list of responses into a single dataframe
df_for_sale = transform_listings_to_df(for_sale_response_list)
print('Num of rows:', len(df_for_sale))
print('Num of columns:', len(df_for_sale.columns))
df_for_sale.head() # preview first 5 rows

Num of rows: 70
Num of columns: 10


,street_address,bedrooms,bathrooms,list_price,property_type,town,image_url,url,lat,lon
0,"Cond. Puerta de la Bahía, CASH 3 hab...",3 Cuartos,1 Baños,"$110,000",Apt/WalkUp,"Condominio-Puerta De La Bahia , San Juan - San...",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.4473458,-66.0791937
1,COND DIPLOMAT CONDADO,1 Cuartos,1 Baños,"$222,000",Apartamento,"Condominio-Diplomat , San Juan - Condado-Miramar",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.456037,-66.07258
2,Condado Lagoon Villas,1 Cuartos,1 Baños,"$400,000",Apartamento,"Condominio-Condado Lagoon Villas , San Juan - ...",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.4623147,-66.0850006
3,Cond Kingsville EXCELENTE OPORTUNIDAD...,None,1 Baños,"$220,000",Apartamento,"Condominio-Kingsville , San Juan - Condado-Mir...",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.452982,-66.061346
4,Apartment in Best Location- AIRBNB,1 Cuartos,1 Baños,"$279,000",Apartamento,"Condominio-Diplomat , San Juan - Condado-Miramar",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.456037,-66.07258


Step #6 - View contents of dataframe

In [13]:
# view count of property types
df_grp1 = df_for_sale.groupby(['property_type'])['street_address'].count()\
  .reset_index()\
  .rename(columns={'street_address': 'count'})
df_grp1['prct'] = df_grp1['count'] / df_grp1['count'].sum()
print(df_grp1)

fig = px.pie(df_grp1, values='count', names='property_type', 
             title='Count of properties by type')
fig.show()

  property_type  count      prct
0   Apartamento     61  0.871429
1    Apt/WalkUp      3  0.042857
2     Comercial      5  0.071429
3         Solar      1  0.014286


In [14]:
# add features
df_for_sale_feat = df_for_sale.copy()
df_for_sale_feat['num_bedrooms'] = df_for_sale_feat.apply(lambda x: 
  int(''.join(e for e in x['bedrooms'] if e.isalnum()).strip()[0]) if x['bedrooms'] != None else 0, axis=1)
df_for_sale_feat['num_bathrooms'] = df_for_sale_feat.apply(lambda x: 
  int(x['bathrooms'][0]) if x['bathrooms'] != None else 0, axis=1)
df_for_sale_feat['list_price_int'] = df_for_sale_feat.apply(lambda x: 
  int(''.join(e for e in x['list_price'] if e.isalnum())), axis=1)
df_for_sale_feat['property_attributes'] = df_for_sale_feat.apply(lambda x: 
  'Bds: {0}, Bths: {1}'.format(str(x['num_bedrooms']), str(x['num_bathrooms'])), axis=1)
df_for_sale_feat.head(1)

,street_address,bedrooms,bathrooms,list_price,property_type,town,image_url,url,lat,lon,num_bedrooms,num_bathrooms,list_price_int,property_attributes
0,"Cond. Puerta de la Bahía, CASH 3 hab...",3 Cuartos,1 Baños,"$110,000",Apt/WalkUp,"Condominio-Puerta De La Bahia , San Juan - San...",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.4473458,-66.0791937,3,1,110000,"Bds: 3, Bths: 1"


In [15]:
# view count of property attributes (bed / bath)
df_grp2 = df_for_sale_feat.groupby(['property_attributes'])['street_address'].count()\
  .reset_index()\
  .rename(columns={'street_address': 'count'})\
  .sort_values(by=['property_attributes'])
df_grp2['prct'] = df_grp2['count'] / df_grp2['count'].sum()
print(df_grp2)

fig = px.pie(df_grp2, values='count', names='property_attributes', 
             title='Count of properties by type')
fig.show()

   property_attributes  count      prct
0      Bds: 0, Bths: 0      8  0.114286
1      Bds: 0, Bths: 1     17  0.242857
2      Bds: 0, Bths: 2      2  0.028571
3      Bds: 0, Bths: 3      1  0.014286
4      Bds: 1, Bths: 1     14  0.200000
5      Bds: 1, Bths: 2      1  0.014286
6      Bds: 2, Bths: 1      5  0.071429
7      Bds: 2, Bths: 2      7  0.100000
8      Bds: 3, Bths: 1      8  0.114286
9      Bds: 3, Bths: 2      5  0.071429
10     Bds: 4, Bths: 1      1  0.014286
11     Bds: 6, Bths: 2      1  0.014286


In [16]:
# view distribution of bedrooms / bathrooms / price
fig1 = px.histogram(df_for_sale_feat, x="num_bedrooms", title="Distribution of Bedrooms")
fig1.show()

fig2 = px.histogram(df_for_sale_feat, x="num_bathrooms", title="Distribution of Bathrooms")
fig2.show()

fig3 = px.box(df_for_sale_feat, y="list_price_int", title="Box Pot of List Price")
fig3.show()

Step #7 - Export file

In [17]:
# uncomment below to download file
# df_for_sale.to_csv('puerto_rico_listings_for_sale.csv', index=False)
# files.download('puerto_rico_listings_for_sale.csv')

#### <font color="purple">2. Properties For-Rent Long-Term</font> ✌
*In Development - Coming Soon*

#### <font color="purple">3. Properties For-Rent Short-Term</font> 🌴
*In Development - Coming Soon*



### <font color="green">Section #2 - Market Analysis</font> 📈
This section will cover how to derive market statistics to montior market performance.
<br>
*In Development - Coming Soon*

### <font color="green">Section #3 - Visualize Properties</font> 🗺
This section will cover how to visualize property listings on a map.

In [18]:
# visualize for-sale listings

# prepare dataframe
df_for_sale_map = df_for_sale_feat.copy()
df_for_sale_map['latitude'] = pd.to_numeric(df_for_sale_map.lat, errors='coerce')
df_for_sale_map['longitude'] = pd.to_numeric(df_for_sale_map.lon, errors='coerce')

# Create a geopandas dataframe from a regular dataframe
gdf = gpd.GeoDataFrame(df_for_sale_map, geometry=gpd.points_from_xy(
    df_for_sale_map.longitude, df_for_sale_map.latitude))

gdf.head(2)

,street_address,bedrooms,bathrooms,list_price,property_type,town,image_url,url,lat,lon,num_bedrooms,num_bathrooms,list_price_int,property_attributes,latitude,longitude,geometry
0,"Cond. Puerta de la Bahía, CASH 3 hab...",3 Cuartos,1 Baños,"$110,000",Apt/WalkUp,"Condominio-Puerta De La Bahia , San Juan - San...",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.4473458,-66.0791937,3,1,110000,"Bds: 3, Bths: 1",18.447346,-66.079194,POINT (-66.07919 18.44735)
1,COND DIPLOMAT CONDADO,1 Cuartos,1 Baños,"$222,000",Apartamento,"Condominio-Diplomat , San Juan - Condado-Miramar",https://imgcache.clasificadosonline.com//PP/FS...,https://www.clasificadosonline.com/UDRealEstat...,18.456037,-66.07258,1,1,222000,"Bds: 1, Bths: 1",18.456037,-66.072580,POINT (-66.07258 18.45604)


In [19]:
# create Kepler map
map_config = get_kepler_map_config()
kepler_map = KeplerGl(height=400, data={'data_1': gdf}, config=map_config)
kepler_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'config': {'mapState': {'bearing': 0, 'dragRotate': False, 'isSplit': False, 'latitude': 18.4…

# End Notebook